In [1]:
import re
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
#LIRE LE DATASET

test = pd.read_csv('Dataset/test.csv')
print("Test Dataset Lu")




#PRETRAITEMENT DU DATASET

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def nettoyer(text):
    text = text.lower()
    # Supprimer les caractères spéciaux
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenisation
    words = nltk.word_tokenize(text)
    # Suppression des mots vides et lemmatisation
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    # Reconstituer le texte prétraité
    return ' '.join(words)
test['processed_text'] = test['text'].apply(nettoyer)
# Convertir en format de corpus TF-IDF (liste de documents)
corpus = test['processed_text'].tolist()
# Vérification
print("Corpus après prétraitement :\n", corpus[:5])

Test Dataset Lu
Corpus après prétraitement :
 ['im feeling rather rotten im ambitious right', 'im updating blog feel shitty', 'never make separate ever want feel like ashamed', 'left bouquet red yellow tulip arm feeling slightly optimistic arrived', 'feeling little vain one']


In [3]:
#VECTORISATION TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer
# Initialiser le vectoriseur TF-IDF
vectorizer = TfidfVectorizer()
# Appliquer la transformation sur le corpus
tfidf_matrix = vectorizer.fit_transform(corpus)
# Afficher la matrice TF-IDF
print("Matrice TF-IDF :\n", tfidf_matrix.toarray())
# Afficher les termes (vocabulaire) et les indices associés
print("\nVocabulaire :\n", vectorizer.vocabulary_)
# Facultatif : Vérifiez les dimensions de la matrice
print("\nDimensions de la matrice TF-IDF :", tfidf_matrix.shape)

Matrice TF-IDF :
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

Vocabulaire :
 {'im': 1891, 'feeling': 1413, 'rather': 3033, 'rotten': 3215, 'ambitious': 131, 'right': 3194, 'updating': 4033, 'blog': 419, 'feel': 1412, 'shitty': 3375, 'never': 2527, 'make': 2298, 'separate': 3326, 'ever': 1298, 'want': 4128, 'like': 2198, 'ashamed': 220, 'left': 2168, 'bouquet': 456, 'red': 3083, 'yellow': 4275, 'tulip': 3946, 'arm': 202, 'slightly': 3460, 'optimistic': 2638, 'arrived': 208, 'little': 2222, 'vain': 4057, 'one': 2626, 'cant': 543, 'walk': 4120, 'shop': 3381, 'anywhere': 172, 'uncomfortable': 3978, 'felt': 1417, 'anger': 145, 'end': 1240, 'telephone': 3772, 'call': 529, 'explain': 1338, 'clung': 690, 'relationship': 3109, 'boy': 460, 'many': 2313, 'way': 4147, 'immature': 1896, 'uncommitted': 3979, 'despite': 1006, 'excitement': 1319, 'getting': 1597, 'accepted': 19, 'master': 2

In [5]:
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out()) # Convertir la matrice TF-IDF en DataFrame
# Sauvegarder le DataFrame en CSV
tfidf_df.to_csv('matrice_tfidf.csv', index=False)

In [7]:
#REPRESENTATION AVEC Word2Vec

corpus_word2vec = [doc.split() for doc in corpus]# Préparer le corpus en liste de listes de tokens pour Word2Vec
# Affichage de vérification
print("Corpus pour Word2Vec (premières phrases) :\n", corpus_word2vec[:5])

#CHARGER LE MODELE Word2Vec PRE-ENTRAINE
from gensim.models import KeyedVectors
word2vec_model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

#CALCULER LES VECTEURS MOYENS POUR CHAQUE PHRASE
def get_sentence_vector(sentence, model):
    word_vectors = [model[word] for word in sentence if word in model]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(model.vector_size)  # Retourne un vecteur nul si aucun mot n'est dans le modèle

# Appliquer la fonction sur chaque phrase du corpus
sentence_vectors = [get_sentence_vector(sentence, word2vec_model) for sentence in corpus_word2vec]
# Vérification de la dimension de chaque vecteur de phrase
print("Vecteur de la première phrase :\n", sentence_vectors[0])
print("\nDimensions de chaque vecteur de phrase :", len(sentence_vectors[0]))

#SAUVEGARDER LES VECTEURS EN CSV
sentence_vectors_df = pd.DataFrame(sentence_vectors)
sentence_vectors_df.to_csv('vectors_word2vec.csv', index=False)

Corpus pour Word2Vec (premières phrases) :
 [['im', 'feeling', 'rather', 'rotten', 'im', 'ambitious', 'right'], ['im', 'updating', 'blog', 'feel', 'shitty'], ['never', 'make', 'separate', 'ever', 'want', 'feel', 'like', 'ashamed'], ['left', 'bouquet', 'red', 'yellow', 'tulip', 'arm', 'feeling', 'slightly', 'optimistic', 'arrived'], ['feeling', 'little', 'vain', 'one']]
Vecteur de la première phrase :
 [ 4.20619436e-02  7.50383660e-02  2.97502782e-02  1.65457591e-01
 -1.53494701e-01  7.99386129e-02  6.07212596e-02 -6.80803582e-02
  4.24107127e-02  8.27287957e-02 -1.08398438e-01 -1.40550882e-01
 -6.86841682e-02 -1.89034604e-02 -7.84301758e-03  1.01178847e-01
 -1.46484375e-03  2.20092773e-01  5.76171875e-02  1.88337057e-03
 -7.16291144e-02  1.36614114e-01  1.45089284e-01  6.73479363e-02
  7.30154887e-02 -8.70884508e-02 -1.05398998e-01  5.33970408e-02
  7.47462688e-03 -6.21817447e-02  3.49295475e-02  9.04366598e-02
  2.79017870e-04 -2.80761719e-03 -1.41470775e-03  1.26063749e-01
 -3.801618

GLOVE

In [8]:
# CHARGER LES VECTEURS PRÉ-ENTRAÎNÉS GloVe
def load_glove_embeddings(file_path):
    embeddings = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

# Charger les vecteurs GloVe (300 dimensions)
glove_file = 'glove.6B/glove.6B.300d.txt'
glove_embeddings = load_glove_embeddings(glove_file)
print(f"Nombre de mots chargés dans les embeddings GloVe : {len(glove_embeddings)}")

# CALCULER LES VECTEURS MOYENS POUR CHAQUE PHRASE
def get_sentence_vector(sentence, embeddings, vector_size=300):
    words = sentence.split()
    word_vectors = [embeddings[word] for word in words if word in embeddings]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(vector_size)  # Retourne un vecteur nul si aucun mot n'est dans le modèle

# Appliquer la fonction à chaque phrase
sentence_vectors_glove = [
    get_sentence_vector(sentence, glove_embeddings, vector_size=300) for sentence in corpus
]

# Vérification des dimensions
print("Vecteur de la première phrase avec GloVe :\n", sentence_vectors_glove[0])
print("\nDimensions de chaque vecteur de phrase :", len(sentence_vectors_glove[0]))

# SAUVEGARDER LES VECTEURS EN CSV
sentence_vectors_glove_df = pd.DataFrame(sentence_vectors_glove)
sentence_vectors_glove_df.to_csv('vectors_glove.csv', index=False)
print("Les vecteurs GloVe ont été sauvegardés dans 'vectors_glove.csv'")

Nombre de mots chargés dans les embeddings GloVe : 400000
Vecteur de la première phrase avec GloVe :
 [ 8.22911412e-02 -2.11954519e-01  5.07264249e-02 -3.41340035e-01
  9.17484984e-02 -8.66639987e-02  2.62185931e-03  7.57014230e-02
 -1.52484283e-01 -1.03926003e+00  1.59382150e-01 -4.98885699e-02
 -3.47209603e-01 -1.10913120e-01 -2.11782426e-01 -2.48442888e-02
 -1.24978714e-01  7.18581453e-02  3.15548569e-01  2.98757136e-01
  1.60007715e-01  2.33070001e-01  2.31542423e-01 -1.40639972e-02
 -2.26584245e-02  6.97242767e-02  2.32755572e-01  2.19342858e-02
  6.91597238e-02 -4.13774326e-02 -1.45985680e-02  5.35767153e-02
  1.72758494e-02 -2.18998000e-01 -6.58101439e-01  5.55785634e-02
 -1.11497149e-01 -1.21362433e-01 -1.52258277e-01  5.85050173e-02
  1.34829000e-01 -9.82728451e-02 -6.89401254e-02 -1.50128707e-01
  2.43878707e-01  2.28341848e-01  1.17558427e-01 -1.32561192e-01
 -1.14761442e-02 -1.43802553e-01  5.99902868e-02 -8.52620006e-02
  2.51929134e-01  5.83245791e-02  5.90299964e-02  4.8

FAST TEXT

In [10]:
from gensim.models import KeyedVectors

# CHARGER LES VECTEURS PRÉ-ENTRAÎNÉS FASTTEXT
# Chemin vers le fichier binaire pré-entraîné (exemple : cc.en.300.bin)
fasttext_model_path = 'wiki-news-300d-1M.vec'

# Charger le modèle FastText avec gensim
print("Chargement du modèle FastText...")
fasttext_model = KeyedVectors.load_word2vec_format(fasttext_model_path, binary=False)
print("Modèle FastText chargé avec succès !")

# CALCULER LES VECTEURS MOYENS POUR CHAQUE PHRASE
def get_sentence_vector_fasttext(sentence, model, vector_size=300):
    word_vectors = [model[word] for word in sentence if word in model]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(vector_size)  # Retourne un vecteur nul si aucun mot n'est dans le modèle

# Préparer les vecteurs pour le corpus
sentence_vectors_fasttext = [
    get_sentence_vector_fasttext(sentence, fasttext_model, vector_size=300)
    for sentence in corpus_word2vec
]

# Vérification des dimensions
print("Vecteur de la première phrase avec FastText :\n", sentence_vectors_fasttext[0])
print("\nDimensions de chaque vecteur de phrase :", len(sentence_vectors_fasttext[0]))

# SAUVEGARDER LES VECTEURS EN CSV
sentence_vectors_fasttext_df = pd.DataFrame(sentence_vectors_fasttext)
sentence_vectors_fasttext_df.to_csv('vectors_fasttext.csv', index=False)

Chargement du modèle FastText...
Modèle FastText chargé avec succès !
Vecteur de la première phrase avec FastText :
 [-8.10142830e-02 -5.39857112e-02 -6.80428594e-02 -5.87428547e-02
  6.65571466e-02  4.48571565e-03  1.00714285e-02 -2.59000007e-02
 -4.52857232e-03 -2.07285713e-02 -4.15857136e-02 -5.18285818e-02
  3.43142860e-02 -4.47857119e-02 -2.90428549e-02 -2.08857153e-02
 -1.43571422e-02 -8.41428526e-03  6.65714359e-03 -1.65428575e-02
 -6.10142872e-02 -2.26999987e-02 -2.25714338e-03 -4.56142910e-02
  3.34285735e-03 -2.78428551e-02  1.72714274e-02 -7.85857216e-02
 -3.21428590e-02  5.95000014e-02  7.36714229e-02  7.45571405e-02
 -2.37571429e-02  5.05428612e-02  9.20714289e-02  5.91285750e-02
 -3.98714319e-02  1.74571406e-02 -4.42857202e-03 -3.18857133e-02
  4.24000025e-02  5.72857168e-03 -8.99428576e-02 -1.72857300e-03
  5.92857134e-03  2.80000013e-03 -3.07285748e-02  2.57571451e-02
 -1.35428561e-02 -2.79142875e-02 -1.01142861e-01 -5.75999990e-02
 -6.64599955e-01 -1.08857140e-01  3.81